### Convert Excel File to csv for AnkiDroid import

Author: Mario Mañana   
Last update: 28/8/2024

In [43]:
import pandas as pd

In [44]:
def excel_to_csv(excel_file, sheet_name, columns, csv_file, ow = True):
    
    # Define converters for the specified columns to ensure they are read as text
    converters = {col: str for col in columns}
        
    # Read the specified sheet from the Excel file
    df = pd.read_excel(excel_file, sheet_name=sheet_name, converters=converters, header=0)
    
    # Rename columns to string if they are numeric
    df.columns = [str(col) for col in df.columns]
    
    # Ensure the specified columns are read as text
    for col in columns:
        df[col] = df[col].astype(str)
    
    #print( df.columns)
       
    # Select the specified columns
    df_selected_columns = df[columns]
    
    if ow == True:
        ow_file = 'w'
    else:
        ow_file = 'a'
    
    # Save the DataFrame to a CSV file with semicolon as the separator
    df_selected_columns.to_csv(csv_file, sep=';', index=False, na_rep='  ', mode=ow_file, header=False)
    
    return( df)



In [45]:
baggregated = True
boverwrite = False 

# Example usage
#path = 'E:\\mario\\UC24\\documentacion\\'
path = 'E:\\mario\\trabajos2\\innovación_docente_2024\\documentacion\\'
excel_file = 'uc_tc.xlsx'  # Path to your Excel file
#sheet_names = ['eadmi', 'presupuesto', 'infraestructuras', 'LOSU', 'academico', 'CifrasUC']        # Name of the sheets to read
#sheet_names = ['eadmi']        # Name of the sheets to read
#sheet_names = ['intro', 'ca', 'resolucion', 'trifasica', 'transitorio', 'cuadripolos', 'bobinas', 'filtros', 'rns']
sheet_names = ['intro', 'ca']
#csv_files =  ['eadmi_00.csv']      # output CSV file
asignatura = 'G822'
columns = ['slide', 'pregunta', 'imagen_frontal', 'respuesta', 'imagen_respuesta', 'r1', 'r2', 'r3', 'r4', 'r5', 'r6', 'r7', 'r8', 'r9', 'r10', 'tags','deck',asignatura] # List of columns to read


print( columns)

csv_aggregated = asignatura + '.csv'

full_excel_path = path + excel_file
print("******************************************")
print(" Input File")
print( full_excel_path)
print("  ")


if baggregated == True:
    full_csv_output = path + csv_aggregated
else:
    full_csv_output = path + sheet + '_00.csv'
print('Output file: ' + full_csv_output)


contenido = """#deck column:17
#separator:Semicolon
#notetype:Basic_Image_several_answers
#columns:slide;pregunta;imagen_frontal;respuesta;imagen_respuesta;r1;r2;r3;r4;r5;r6;r7;r8;r9;r10;tags;deck
#tags column:16
#html:true
"""
# Escribir (sobrescribir si ya existe) el archivo
with open( full_csv_output, "w") as archivo:
    archivo.write(contenido)



for index, sheet in enumerate( sheet_names):
    print("-------------------")
    print("Sheet: " + sheet)
    
    
    df = pd.read_excel( full_excel_path, sheet_name=sheet, usecols=columns, header=0, dtype=str)
    df.columns = df.columns.astype( str)
    
    df2 = df[ df[asignatura].notna()]
       
    for index, row in df2.iterrows():
        df2.at[ index, 'deck'] = asignatura + '::' + 'Tema' + str(row[asignatura])  
        df2.at[ index, 'slide'] = 'sheet' + '.' + str(row['slide'])
    
    dfs = df2.drop( columns=[asignatura])
    dfs.to_csv( full_csv_output, sep=';', index=False, na_rep='  ', mode='a', header=False)
       
       
    #df = excel_to_csv( full_excel_path, sheet, columns, full_csv_output, ow=boverwrite)
    print(dfs)
    


['slide', 'pregunta', 'imagen_frontal', 'respuesta', 'imagen_respuesta', 'r1', 'r2', 'r3', 'r4', 'r5', 'r6', 'r7', 'r8', 'r9', 'r10', 'tags', 'deck', 'G822']
******************************************
 Input File
E:\mario\trabajos2\innovación_docente_2024\documentacion\uc_tc.xlsx
  
Output file: E:\mario\trabajos2\innovación_docente_2024\documentacion\G822.csv
-------------------
Sheet: intro
     slide                        pregunta imagen_frontal  \
1  sheet.2  Unidad de la potencia reactiva            NaN   

                                           respuesta imagen_respuesta   r1  \
1  voltiamperio reactivo (en inglés reactive volt...              NaN  NaN   

    r2   r3   r4   r5   r6   r7   r8   r9  r10    tags         deck  
1  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  #intro  G822::Tema2  
-------------------
Sheet: ca
     slide                                           pregunta imagen_frontal  \
0  sheet.1  ¿A qué frecuencia se denomina <b>de marcha ind...            N

In [46]:
df

,slide,pregunta,imagen_frontal,respuesta,imagen_respuesta,r1,r2,r3,r4,r5,r6,r7,r8,r9,r10,tags,deck,G822
0,1,¿A qué frecuencia se denomina <b>de marcha ind...,NaN,50 Hz,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,#ca,NaN,1
